In [10]:
# Import Required Libraries
import sys
import os
import datetime
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferWindowMemory

# Import the actual Source model for correct typing
from typing import List
from datetime import datetime
from app.models.chat_models import Source

# Demonstrating LangChain Usage in RAGService

This notebook demonstrates how LangChain is utilized in the `RAGService` class for AI chat functionality with context retrieval.

In [11]:
# Initialize a Mock VectorStoreService
class MockVectorStoreService:
    async def search_similar_notes(self, query: str, n_results: int):
        return [
            Source(
                note_id=1,
                title="Note 1", 
                content_snippet="This is a snippet of note 1 containing important information about AI and machine learning concepts.",
                relevance_score=0.9,
                created_at=datetime.now(),
                updated_at=datetime.now()
            ),
            Source(
                note_id=2,
                title="Note 2", 
                content_snippet="This is a snippet of note 2 with details about software development and programming best practices.",
                relevance_score=0.8,
                created_at=datetime.now(),
                updated_at=datetime.now()
            )
        ]

mock_vector_store_service = MockVectorStoreService()

In [12]:
# Create a RAGService Instance
from app.services.rag_service import RAGService

rag_service = RAGService(vector_store_service=mock_vector_store_service)

# Mock initialization (skipping actual API key setup for demonstration)
import asyncio
await rag_service.initialize()

In [13]:
# Simulate a Chat Interaction
async def simulate_chat():
    user_message = "What is the summary of note 1?"
    response = await rag_service.chat(message=user_message)
    print("Assistant Response:", response.response)
    print("Sources used:", len(response.sources))
    for i, source in enumerate(response.sources):
        print(f"  {i+1}. {source.title} (score: {source.relevance_score:.2f})")

await simulate_chat()

Assistant Response: Note 1 provides important information about AI and machine learning concepts. It covers essential topics related to artificial intelligence and machine learning, offering a foundational understanding of these technologies. If you have any specific questions or need more details from Note 1, feel free to ask!
Sources used: 2
  1. Note 1 (score: 0.90)
  2. Note 2 (score: 0.80)


In [14]:
# Inspect System Prompt Generation
sources = [
    Source(
        note_id=1,
        title="Note 1", 
        content_snippet="This is a snippet of note 1 about AI and machine learning.",
        relevance_score=0.9,
        created_at=datetime.now(),
        updated_at=datetime.now()
    ),
    Source(
        note_id=2,
        title="Note 2", 
        content_snippet="This is a snippet of note 2 about software development.",
        relevance_score=0.8,
        created_at=datetime.now(),
        updated_at=datetime.now()
    )
]
system_prompt = rag_service._create_system_prompt(sources)
print("Generated System Prompt:\n", system_prompt)

Generated System Prompt:
 You are an AI assistant helping users find and discuss information from their personal notes. 

Your role:
- Answer questions based on the provided note context
- Be helpful, accurate, and conversational
- If the context doesn't contain enough information, say so honestly
- Always cite which notes you're referencing when possible
- Summarize and synthesize information from multiple notes when relevant

Guidelines:
- Be concise but thorough
- Use a friendly, personal tone since these are the user's own notes
- If asked about something not in the notes, clarify that you can only work with the provided notes
- When referencing notes, mention the note title when possible

Relevant notes context:

1. Note: "Note 1"
   Content: This is a snippet of note 1 about AI and machine learning.
   Relevance: 0.90

2. Note: "Note 2"
   Content: This is a snippet of note 2 about software development.
   Relevance: 0.80



In [15]:
# Review Conversation History Management
conversation_id = rag_service._get_or_create_conversation(None)
rag_service._add_message_to_conversation(conversation_id, "user", "What is note 1 about?")
rag_service._add_message_to_conversation(conversation_id, "assistant", "Note 1 is about AI and ML.")

conversation_context = rag_service._get_conversation_context(conversation_id)
print("Conversation Context:", conversation_context)

Conversation Context: [HumanMessage(content='What is note 1 about?'), AIMessage(content='Note 1 is about AI and ML.')]


In [16]:
# Check which LLM is being used
print("=== LLM Information ===")

# Method 1: Check the LLM object directly
if rag_service.llm:
    print(f"LLM Type: {type(rag_service.llm).__name__}")
    print(f"Model Name: {getattr(rag_service.llm, 'model_name', 'Unknown')}")
    print(f"Temperature: {getattr(rag_service.llm, 'temperature', 'Unknown')}")
    print(f"Max Tokens: {getattr(rag_service.llm, 'max_tokens', 'Unknown')}")
else:
    print("LLM not initialized")

print()

# Method 2: Use the service stats method
stats = rag_service.get_service_stats()
print("=== Service Stats ===")
for key, value in stats.items():
    print(f"{key}: {value}")

print()

# Method 3: Check environment variables that control LLM selection
print("=== Environment Configuration ===")
print(f"OPENAI_MODEL: {os.getenv('OPENAI_MODEL', 'Not set (defaults to gpt-3.5-turbo)')}")
print(f"OPENAI_TEMPERATURE: {os.getenv('OPENAI_TEMPERATURE', 'Not set (defaults to 0.7)')}")
print(f"OPENAI_API_KEY: {'Set' if os.getenv('OPENAI_API_KEY') else 'Not set'}")


=== LLM Information ===
LLM Type: ChatOpenAI
Model Name: gpt-3.5-turbo
Temperature: 0.7
Max Tokens: 1000

=== Service Stats ===
total_conversations: 2
total_messages: 4
max_context_notes: 5
conversation_window: 10
model: gpt-3.5-turbo

=== Environment Configuration ===
OPENAI_MODEL: gpt-3.5-turbo
OPENAI_TEMPERATURE: 0.7
OPENAI_API_KEY: Set


# Conclusion

This notebook demonstrated how LangChain is used in the `RAGService` class for AI chat functionality, including system prompt generation, conversation history management, and interaction with a mock LLM.